<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Firmware Reference

This a Jupyter Notebook Reference document for the SLab projects

Version 1.1 (6/3/2019) License information is at the end of the document

---

This document describes the firmware that runs on the **Hardware Board** 

<a id='system'></a>

<div class="alert alert-block alert-info">
<BR>
<font size="10"> Basics </font> 
<BR>
</div>

The **SLab system** is mainly composed of four elements

* A hardware board that holds a microcontroller unit (MCU)
* Firmware code inside the board MCU
* Python code on the PC
* Documentation files on the PC

This document describes the **firmware** contained in the **hardware board**

## Protocol basics

The system operates using the **client-server** model.

The **hardware board**, whose firmware is described in this document, is the **server** which waits from client requests.

The **PC**, that hosts the Python code, is the client.

When the board boots, it performs the initializations that will be described later and waits for client requests.

The communication is based on **commands** that are started by the client PC. The protocol is simple: The client PC sends a **command code** followed by a **PC payload** and waits for a response from the board. 

The client receives the command code and the payload, interprets its contents and check its integrity, performs any requested operation and responds to the PC with a **MCU payload**. The communication ends when the PC receives the payload and checks its integrity. 

![Client-Server](images/Firmware/client_server.png)


From the server MCU point of view the command starts when the **comand code** is received and ends when the **MCU payload** is sent.

From the client PC point of view the command starts when it sends the **command code** and ends when it receives the **MCU** payload.

## CRC

All transmissions, except for some particular cases, are protected by **CRC** checks. The CRC for a sequence of transmitted bytes is a byte obtained by Xoring all Bytes.

Both the **server** and the **client** compute the **CRC** for the data they transmit (Tx) and receive (Rx). The **Tx CRC** is sent as the last byte in a transmission ending the payload. The **Rx CRC** is received as the last byte in reception and is used, by the other side of the link, to check the **CRC** of the received data. If the computed **Rx CRC** is different from the received **CRC**, a **CRC Error** is generated.

All bytes associated to a command, including the **command code** are used for the **CRC** calculation.

The actions taken when there is a **CRC Rx Error** are different in the MCU server and the client PC.

If the MCU server detects a mismatch between the received data **Rx CRC** and the **CRC** the PC sends after the **PC payload**, it sends the following **MCU payload** after ending the command and wait for a new command.

* An **ECRC** Byte
* The **Tx CRC**

When this **payload** is received by the PC Python code, and **exception** with the message **"CRC Error in PC to Board link"** is generated.

If the PC Python client detects a CRC mismatch, it generates an **exception** with the message: **"CRC Error in Board to PC link"**

In both cases the command ends

## Data objects

In both directions of the serial link, there are four kind of data objects that can be sent:

**Bytes**  
Definded as unsigned 8 bit integers  

**Chars**  
Coded in ASCII on a Byte

**Strings**  
Sequences of Chars  

**Words**  
Defined as unsigned 16 bits integers  
Communication is **little endian** so the least significant Byte is sent first.

**Floats**  
SLab communications uses a custom 24 bit (3 Byte) float type.  
Most firmware operations are carried out using integers. Float values are only used for parameters, like the sample time, that feature a high dynamic range.

The 24 bit float contains a 16 bit **mantisa** and a 8 bit **exponent**.  
The mantisa is coded as a 16 bit integer with a 20000 value offset
The exponent is coded as a 8 bit integer with a 128 offset
The float number can be calculated:

$\qquad Float = (mantisa - 20000) \cdot 10^{\;exp - 128}$

Note that the exponent is base 10, not base 2, and that the format has several ways to code the same numbers

During the transmission the exponent is sent first and the **mantisa** is sent afterwards as **little endian**

* Byte 1 : Exponent
* Byte 2 : Low mantisa
* Byte 3 : High mantisa

## Board start

When the board starts, the hardware is initialized, inluding the serial communication.

Then the **Firmware String** is sent to the serial port.

The firmware string describes the board and the firmware, including its version. An example can be something like:

$\qquad$ **"Nucleo64-F303RE ChibiOS SLab2 v2.0\r\n"**

That way, if you connect the board and open the serial link using a terminal application, you get the string when you reset the board. That means that you don't need to use Python to check that the board is able to boot.

After showing this string, the board performs a **Soft Reset** and starts the server operation waiting for commands from the PC.


## Soft Reset

A **Soft Reset** is performed each time the board boots due to a **Hard Reset**. It can also be issued, as we will see later, by an specific command.

The soft reset initializes the board state. The operation of the board after a **Soft Reset** doesn't depend on any action taken before the **Soft Reset**

This can be useful to guarentee the repeatibility of some experiments

The **Soft Reset**, at least, performs the following actions:

* Set DACs to zero
* Set number of average DC readings to 10
* Set sample time to 1ms
* Configures transient storage for 1000 samples on ADC1
* Erase the Unified Memory Buffer (UMB)
* Set all digital I/O lines to input with pull-down

## Unified Memory Buffer (UMB)

Most of the available RAM memory on the hardware board is dedicated to the **Unified Memory Buffer (UMB)**

The UMB holds samples defined as unsigned 16 bit values.

Analog values have a minimum voltage of OV and a maximum typically equal to the board Vdd. The minimum voltage is coded with a zero value and the maximum voltage is coded with the maximum unsigned 16 bit number $2^{16}-1$  
This coding is independent of the resolution, in number of bits, of the ADC and DAC converters. For instance, if the converters have 12 bits, then the first 4 bits of the coded values will always be zero.

A board can have a maximum of 16 digital I/O lines. The state of all lines can be stored in a 16 bit word by using one bit for each line.

The UMB can contain, always in this order:

* A primary wavetable (DAC values)
* A secondary wavetable (DAC values)
* A digital wavetable (Digital I/O values)
* Transient measurements (ADC values)

The following figure shows a diagram of the UMB. There are four pointers used for accessing its data. **buff** is the main UMB pointer and accesses the primary wavetable. **wave2buff** accesses the secondary wavetable. **waveDbuff** accesses the digital wavetable. Finally, **tranBuff** accesses the measurement data.

![UMB](images/Firmware/umb.png)

Adding any element erases the ones that follow. For instance, uploading a **primary wavetable** erases all the other elements. You can, however, change the **secondary wavetable** without affecting the primary one.

After uploading the wavetables, all the unused UMB space is available for measurements.

Transient measurement commands can store measuremens of **na** analog channels and/or **nd** digital lines. Analog measurements require one word for each sample on each ADC channel. Digital measurements always require a word for each sample regardless of the number of I/O lines.

Although the measurement information of the different ADC channels and the digital lines is usually stored  interleaved, the data is always sent, as we will see later, by sending together all the samples of each channel, followed by the digital samples.

The UMB analog data always contains RAW uncalibrated ADC and DAC values. All the calibrations are performed at the PC side.

<a id='system'></a>

<div class="alert alert-block alert-info">
<BR>
<font size="10"> Commands </font> 
<BR>
</div>

For each command the following information is provided:

* Description of the command
* Request sent from the PC to the MCU
* Response sent from the MCU to the PC
* Example of slab Python command(s) that use this firmware command

In the **request** and **response** descriptions we understand that there is always a receive for each send in the other side of the link, so those complementary actions won't be described.

Every **Send CRC** action is associated to a **Receive and check CRC** that is not shown.

The previous **CRC** section describes the actions taken by the PC or the MCU upon a CRC mismatch. In all command we will only describe the actions taken when there are no CRC errors.

In most commands, the MCU sends an acknowledge (ACK) Byte after the **PC payload** is received as the first Byte of the **MCU payload**. Sometimes, the MCU checks the **PC payload**. If the CRC matches but the payload contains incorrect values, like trying to set an inexistent DAC number, a **NACK action** takes place with the following actions:

* Send Byte NACK
* Send Byte CRC
* End the command

After the PC sends its payload, it usually waits for the ACK Byte and operates depending on what really receives:

* If ACK is received, continue with the command
* If ECRC is received, raises a **"CRC Error in PC to Board link"** exception
* If NACK is received, raises a **"Remote Error : Bad command parameters"** exception
* Otherwise, raises a **"Unknown Board Response"** exception

# Command List

To ease the search for topic, you can use the following list of **hyperlinks**

**Board information**

[Get Firmware String (F)](#GetFirmwareString)  
[Get Magic (M)](#GetMagic)  
[Get Board Information (I)](#GetBoardInformation)  
[Send Pin List (L)](#SendPinList)  

**Analog DC actions**

[Set Number of Readings (N)](#SetNumberOfReadings)  
[ADC Read (A)](#ADCRead)  
[DAC Write (D)](#DACWrite)  

**Basic transient actions**

[Set Sample Time (R)](#SetSampleTime)  
[Set Storage (S)](#SetStorage)  
[Async Read (Y)](#AsyncRead)  
[Triggered Read (G)](#TriggeredRead)  
[Step Response (P)](#StepResponse)  

**Wavetable upload**

[Load Wavetable (W)](#LoadWavetable)  
[Load Secondary Wavetable (w)](#LoadSecondaryWavetable)  
[Load Digital Wavetable (O)](#LoadDigitalWavetable)  

**Wave response**

[Wave Response (V)](#WaveResponse)  
[Dual Wave Response (v)](#DualWaveResponse)  
[Single Wave Response (X)](#SingleWaveResponse)  

**Wave Play**

[Wave Play (Q)](#WavePlay)  
[Dual Wave Play (q)](#DualWavePlay)  

**Digital I/O**

[Digital I/O Mode (H)](#DigitalIOMode)  
[Digital I/O Write (J)](#DigitalIOWrite)  
[Digital I/O Read (K)](#DigitalIORead)  
[Digital I/O Write All (j)](#DigitalIOWriteAll)  
[Digital I/O Read All (k)](#DigitalIOReadAll) 

**Code Constants**

[Code Constants](#CodeConstants)  

Now follows the list of firmware commands

<a id='GetFirmwareString'></a>

## Get Firmware String (F)

Request a string that describes the firmware

This is the only command that don't use **CRC**. No CRC is calculated or verified in the PC or the MCU.

The string sent is the same string the board sends to the serial port after a **Hard Reset**

>PC  
>* Send Char : 'F'  

>MCU  
>* Send String terminated on CR + LF  

**Example:** connect

<a id='GetMagic'></a>

## Get Magic (M)

Request the magic code

The magic code is requested by the PC to check that there is a compatible SLab board on the other side of the serial link. It is used, for instance, for serial port autodetection.

>PC
>* Send Char : 'M'
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send 4 Byte **Magic Code**
>* Send Byte CRC

**Example:** connect

<a id='GetBoardInformation'></a>

## Get Board Information (I)

Get information about the board and its capabilities. 

Care has been taken so that the user does not need to configure anything inside SLab when using different boards or firmwares.

This command should give the user and the SLab Python code enough information to make the best use of the board features.

>PC
>* Send char : 'I'
>* Send Byte CRC

>MCU
>* Check CRC
>* Send Byte ACK
>* Send Byte with number of board DACs
>* Send Byte with number of board ADCs
>* Send Word with UMB size (in 16 bit samples)
>* Send Float with maximum sample time (in seconds)
>* Send Float with minimum sample time (in seconds)
>* Send Float with board Vdd (in Volt)
>* Send Float with MSFFR (in Hz)
>* Send Float with board Vref (in Volt)
>* Send Byte with DAC number of bits
>* Send Byte with ADC number of bits
>* Send Byte with number of digital I/O lines
>* Send Byte with **Reset State**
>* Send Byte CRC

The **MSFFR** is the Maximum Sample Frequency for Frequency Response. It is a recommendation for the maximum sample frequency to use for sine wave generation in frequency response commands.

The **Reset State** Byte is non zero if the board has not changed state from the previous **Soft Reset**. It is zero if a command with **side effects** has been executed since the previous  **Soft Reset**.

The SLab PC code checks that the **MCU payload** has the expected size. If there is more data than expected, it only reads the known first part of the payload and the CRC is not checked.

**Example:** connect

## Get Extra Information (i)

Get additional information about the board and its capabilities

The previous (I) command was not defined to hold a vatiable content size. This extra command expands the previous command capabilities withou breaking the backward compatibility. 

>PC
>* Send char : 'i'
>* Send Byte CRC

>MCU
>* Check CRC
>* Send Byte ACK
>* Send Word **InfoVersion**
>* **Send Information**
>* Send Byte CRC

The **Send Information** stage depends on the value of the **InfoVersion** field

For **InfoVersion** 0 and up, it sends:

* Send Byte **naac** Number of analog channels for AC

**Example:** connect

<a id='SendPinList'></a>

## Send Pin List (L)

Request the pin list from the board

The pin list contains the pin names of the board DACs, ADCs and digital I/O lines.

>PC
>* Send Char 'L'
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send String PIN_LIST
>* Send Byte CRC

The **PIN_LIST** string can be something like:

$\qquad$ "PA4|PA5|PA1|PB0|PB11|PA0|PA7|PB14|PB13|PC0|PC5|PC6|PC7|PC8|PC9|PC10|PC11|PC12|$"

It contains a sequence of pin names. Each name ends with the '|' char. After the last name, a '$' char is included.

The list contains this ordered list of pins:

* DAC pins, starting with DAC1
* ADC pins, starting with ADC1
* Digital I/O pins, starting with DIO0

In order to properly decode the pin list, the PC needs to know the number of pins of each kind. This information can be obtained with the **Get Board Information (I)** command.

**Example:** connect

## Soft Reset Command (E)

Generates a **Soft Reset** on the board

>PC
>* Send Char 'E'
>* Send Byte CRC 

>MCU
>* Send Byte ACK
>* Send Byte CRC

The command performs the actions described in the previous described **Soft Reset** procedure.

The board is put in a known reset state

**Example:** softReset

<a id='SetNumberOfReadings'></a>

## Set Number of Readings (N)

Set the number of readings to average in an ADC read

>PC
>* Send Char 'N'
>* Send Word **nread**
>* Send CRC

>MCU
>* Send Byte ACK
>* Send Byte CRC

The command just stores the **nread** value in a variable that will be accessed by the followind **ADC Read** command

**Example:** setDCreadings

<a id='ADCRead'></a>

## ADC Read (A)

Reads an ADC channel

>PC
>* Send Char 'A'
>* Send Byte **Channel**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Word **Reading**
>* Send Byte CRC

The **Channel** Byte is the number of ADC to read. If the number is out of range, a **NACK action** is performed.

To obtain the **Reading** the MCU performs the following actions:

* Uses the pin multiplexer to connect the ADC pin to one available ADC peripheral
* Obtains and discards one ADC reading
* Performs **nread** reads on the ADC
* Obtains the mean of the **nread** readings

**Example:** readVoltage

<a id='DACWrite'></a>

## DAC Write (D)

Writes on a DAC channel

>PC
>* Send Char 'D'
>* Send Byte **Channel**
>* Send Word **Value**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Byte CRC

The **Channel** Byte is the number of DAC to set. If the number is out of range, a **NACK action** is performed.

The **Value** Word is the value to set on the DAC.

**Example:** setVoltage

<a id='SetSampleTime'></a>

## Set Sample Time (R)

Sets the sample period **Ts** for the following transient measurements.

>PC
>* Send Char 'R'
>* Send Float **Ts**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Byte CRC

The **Ts**, in seconds, is the time between samples in transient measurements. If the value is out of range, a **NACK action** is performed.

![UMB](images/Firmware/ts.png)

**Example:** setSampleTime

<a id='SetStorage'></a>

## Set Storage (S)

Sets the storage to be used in the following transient measurements.

>PC
>* Send Char 'S'
>* Send Byte **na**
>* Send Byte **nd**
>* Send Word **ns**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Byte CRC

The Byte **na** defines the number of ADC channels to read at each sample time.

The Byte **nd** defines the number of digital inputs to read at each sample time.

The Word **ns** defines the number of samples to capture. As the sample period is **Ts**, the measurement time can be calculated:

$\quad Tm = ns \cdot Ts$

If any value is out of range, a **NACK action** is performed.

**Example:** setTransientStorage

<a id='AsyncRead'></a>

## Async Read (Y)

Performs an asynchronous read

After the **PC payload** is received, the MCU captures **ns** samples separated by **Ts** intervals. When the capture ends, the data is sent to the PC.

>PC
>* Send Char 'Y'
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Perform the measurement
>* Send Byte **Tran Status**
>* Send **Tran Measurement**
>* Send Byte CRC

**Measurement**

The measurement is performed by defining a Word pointer at the first free sample position of the UMB memory and a sample counter initialized to **ns**.

Then we activate the periodic execution of a **callback** function associated to a timer. This function is executed at each sample interval.  
If **na** is not zero, reads the **na** ADCs and stores the readings in the UMB.
If **nd** is not zero, reads all digital I/O lines and stores them as a sample in the UMB.  
After each sample is stored the UMB pointer is increased.  
After storing all samples, the sample counter is decreased.

The Digital estate of all I/O lines is stored on a single 16 bit sample. Each DIO line status is stored in one bit of the sample. All lines are read regardless of their input or output configuration.

When the sample counter reaches zero, we stop the measurement.

![Async Read](images/Firmware/async.png)

The total measurement time $T_M$ can be calculated:

$\qquad T_M = T_S \cdot n_S$

During the measurement, the firmware checks that the **callback** is able to return before the next sample interval. If this condition fails, a **sample overrun** condition is registered.

**Tran Status**

After the measurement is performed, a **Tran Status** Byte is sent. This code can have one of the following values:

* TRAN_OVERRUN if a **sample overrun** condition was registered
* TRAN_HALT if the measurement was halted by the **halt** button
* TRAN_OK if the measurement was completed succesfully

In the first two cases, the **Send Tran Measurement** is skipped.

**Send Tran Measurement**

After **Tran Status** code TRAN_OK is sent, the measurement data is sent to the PC. This corresponds to the following sequence of actions:

* Send Byte **na**
* Send Byte **nd**
* Send Word **ns**
* Send **analog data**
* Send **digital data**

Note that the **na**, **nd** and **ns** values are sent to the PC although they should be known by it. This way we don't require the PC code to keep track of values that are set by other commands. Moreover, it makes the system more robust to reconnections.

The analog data is sent channel by channel. For instance, if **na** is 2, **ns** samples are sent associated to **ADC1** followed by **ns** samples associated to **ADC2**

The digital data is sent when **nd** is not zero. It always contain **ns** 16 bit samples.

**Example:** transientAsync

<a id='TriggeredRead'></a>

## Triggered Read (G)

Performs a triggered read

After the **PC payload** is received, the MCU wait for a trigger condition on **ADC1** and captures **ns** samples separated by **Ts** intervals arround this point: **ns**/2 samples are captured before the trigger condition and the rest are captured afterwards. When the capture ends, the data is sent to the PC.

>PC
>* Send Char 'G'
>* Send Word **Trigger Value**
>* Send Byte **Trigger Mode**
>* Send Byte **Timeout**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Perform the measurement
>* Send Byte **Tran Trig Status**
>* Send **Tran Trig Measurement**
>* Send Byte CRC

The **Trigger Value** is the **ADC1** value, ratiometric 16 bit, that triggers the capture.

The **Trigger Mode** can have two values:

* TMODE_RISE : Trigger when ADC1 crosses the value rising  
* TMODE_FALL : Trigger when ADC1 crosses the value falling  

If the mode is out of range, a **NACK action** is performed.

The command waits for the triggering condition to perform the measurement. It is possible that this condition is never satisfied. That would lock the firmware. There are two ways to **abort** the measurement.

* Use of the **halt** button
* Exceding a **timeout** wait time

The **timeout** parameter can define the number of seconds to wait fot the trigger condition. Measurement will only **abort** if the triggering condition has not yet happened by the time the **timeout** ends. As the **timeout** is a Byte value, the maximum value is 255 seconds. To disable the timeout feature, set the **timeout** Byte to zero.

**Measurement**

The triggered read is the most complex SLab firmware command. It starts defining two counters:

* **presamples** set to $ns/2$
* **postsamples** set to $ns - presamples$

Then, a **pointer** is set to the first free sample position of the UMB memory.

We activate the periodic execution of a **callback** function associated to a timer. This function is executed at each sample interval, and can actuate in four modes, starting in **mode 0**.

In all modes, if **na** is not zero, a sample is always captured and stored, for all **na** enabled ADC channels , in the UMB. If **nd** is not zero, the digital I/O lines are also read and stored as a sample on the UMB.

The UBM if filled in a circular mode. When the captured samples reach **ns**, we reset the **pointer** and we overwrite the previouly captured samples.

![Triggered Read](images/Firmware/triggered.png)

In **mode 0** (presample) the **presamples** counter is decreased. When the counter reaches zero, we change to **mode 1**. This stage guarantees that we always have read, at least, **ns**/2 samples before the triggering condition.

In **mode 1** (precondition)  we wait for **ADC1** reading to be *before* the triggering condition:

* For **rise**, we wait **ADC1** to be below the **Trigger Value**
* For **fall**, we wait **ADC1** to be above the **Trigger Value**

When the condition is met, we jump to **mode 2**

In **mode 2** (postcondition) we wait fot the complementary condition of the previous mode.

* For **rise**, we wait **ADC1** to be above the **Trigger Value**
* For **fall**, we wait **ADC1** to be below the **Trigger Value**

When the condition is met, we store the current UMB pointer position to remember the triggering sample position in the circular buffer and we jump to **mode 3**

In **mode 3** (postsample) the **postsamples** counter is decreased. When the counter reaches zero, we end the measurement. This stage guarantees that we have the required number of samples after the triggering condition.

When the sample counter reaches zero, we stop the measurement. At this point we should have exactly the requested number of samples before and after the triggering condition.

During the measurement, the firmware checks that the **callback** is able to return before the next sample interval. If this condition fails, a **sample overrun** condition is registered.

**Tran Status**

After the measurement is performed, a **Tran Status** Byte is sent. This code can have one of the following values:

* TRAN_OVERRUN if a **sample overrun** condition was registered
* TRAN_HALT if the measurement was halted by the **halt** button
* TRAN_TIMEOUT if a **timeout** takes place before the triggering condition
* TRAN_OK if the measurement was completed succesfully

In the first three cases, the **Send Tran Measurement** is skipped.

**Send Tran Measurement**

After a **Tran Status** code TRAN_OK is sent, the measurement data is sent to the PC. This corresponds to the following sequence of actions:

* Send Byte **na**
* Send Byte **nd**
* Send Word **ns**
* Send **analog data**
* Send **digital data**

As in the **Async Read**, the **na**, **nd** and **ns** values are sent to the PC although they should be known by it. This way we don't require the PC code to keep track of values that are set by other commands. 

Remember that during the wait for the triggering condition,  the UMB was running in circular mode.  We stored a copy of the UMB pointer position at the end of **mode 2** so that we know which samples in the UMB are **presamples** and which are **postsamples**

The firmware first sends the **presamples** to the PC, then the **postsamples**. The PC knows that the sample $ns/2$ corresponds to the triggering point.

**Example:** transientTriggered

<a id='StepResponse'></a>

## Step Response (P)

Obtains the **Step Response** of a circuit

>PC
>* Send Char 'P'
>* Send Word **Step Value**
>* Send CRC

>MCU
>* Send Byte ACK
>* Perform the measurement
>* Send Byte **Tran Status**
>* Send **Tran Measurement**
>* Send Byte CRC

The command operates like the **Async Read** command. The only difference is that, when $ns/5$ samples have elapsed, **DAC1** is set to the **Step Value** sent previously  from the PC.

![Step](images/Firmware/step.png)

Note that the PC gives the **end** value of the step, not the **start** one. This is because the PC sets the **start** value using a **DAC Write** command.

**Example:** stepResponse

<a id='LoadWavetable'></a>

## Load Wavetable (W)

Loads the primary wavetable at the start of the UMB. The primary wavetable is a sequence of sample values to be sent to **DAC1** for wave generation.

>PC
>* Send Char 'W'
>* Send Word **ws**
>* Send **ws** Word samples
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Byte CRC

If the number of wave samples **ws** is bigger than the available UMB space, a **NACK** condition is generated.

If not, the **ws** samples are stored on the low region of the UMB.

The load of the primary wavetable automatically erases the secondary and the digital ones, if they were present on the UMB.

**Example:** loadWavetable

<a id='LoadSecondaryWavetable'></a>

## Load Secondary Wavetable (w)

The secondary wavetable is a sequence of sample values to be sent to **DAC2** for wave generation.

This command is similar to the previous one.  The difference is that the wave is stored after the current primary wavetable. 

>PC
>* Send Char 'W'
>* Send Word **ws2**
>* Send **ws2** Word samples
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Byte CRC

If the number of secondary wave samples **ws2** is bigger than the available UMB espace, taking into account the samples stored for the primary wave, a **NACK** condition is generated.

If not, the **nw2** samples are stored after the primary wave samples. If no primary wave is present, the samples are stored on the low region of the UMB. Note that, in general, it is ilegal to generate waves on **DAC2** without generating also on **DAC1**

The load of the secondary wavetable automatically erases the digital one if it was present on the UMB.

**Example:** loadWavetable with True **second** parameter

<a id='LoadDigitalWavetable'></a>

## Load Digital Wavetable (O)

Loads a digital wavetable

>PC
>* Send Char 'O'
>* Send Word **wd**
>* Send Word **mask**
>* Send **wd** Word samples
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Byte CRC

If the number of digital samples **wd** is bigger than the available UMB espace, a **NACK** condition is generated.

If not, the **wd** samples are stored after the secondary wave samples.

The **mask** word identifies the I/O lines that should be modified, as indicated in the samples, at each sample time

If a digital wavetable has been loaded, digital I/O excitation will be generated in all future wave response and wave play commands. To disable the digital wave generator, the digital wavetable shall be cleared. This can be done in four different ways:

* Performing a **Soft Reset**
* Loading a primary wave
* Loading a secondary wave
* Loading a zero size digital wavetable

Note that in order to generate digital signals you also need to configure the selected I/O lines to an output mode.

**Example:** loadDigitalWavetable

<a id='WaveResponse'></a>

## Wave Response (V)

Obtains the response of a circuit against a waveform generated on **DAC1** and/or the digital I/O lines.

>PC
>* Send Char 'V'
>* Send Word **npre**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Perform the excitation and measurement
>* Send Byte **Tran Status**
>* Send **Tran Measurement**
>* Send Byte CRC

The **excitation and measurement** takes place on two stages

In the **first stage**, **npre** full waves are sent to **DAC1** at one sample on each **sample interval**. Digital samples are also sent if enabled.

If no **primary wave** is defined, **npre** refers to the digital waves.

On the second stage the wave generation continues during **ns** samples for a $T_M$ measurement time. During this stage the ADCs and/or digital I/O lines are read at each **sample interval** like in the **Async Read** command. 

![Wave](images/Firmware/wave.png)

After the secod stage ends, the rest of the command is equal to the **Async Read** one

**Example:** waveResponse

<a id='DualWaveResponse'></a>

## Dual Wave Response (v)

Obtains the response of a circuit against waveforms generated on **DAC1** and **DAC2** and the digital I/O lines if a wavetable was loaded for them.

>PC
>* Send Char 'v'
>* Send Word **npre**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Perform the excitation and measurement
>* Send Byte **Tran Status**
>* Send **Tran Measurement**
>* Send Byte CRC

The comand operates like the previous **Wave Response** one. The only difference is that the **secondary waveform** is also generated on **DAC2**

You should always use the previous **Wave Response** command if you have not uploaded both the **primary** and **secondary** wavetables.

![Wave2](images/Firmware/wave2.png)

If both waves feature a different sample length, they will have different wave frequency and will go out of sync

**Example:** waveResponse with a True **dual** parameter

<a id='SingleWaveResponse'></a>

## Single Wave Response (X)

Obtains the response of a single ADC channel against a waveform generated on **DAC1**

>PC
>* Send Char 'X'
>* Send Byte **nch**
>* Send Word **npre**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Perform the excitation and measurement
>* Send Byte **Tran Status**
>* Send **Tran Measurement**
>* Send Byte CRC

In the previous **Wave Response** and **Dual Wave Response**, a previous call to **Set storage** defines both the **ns** number of samples to capture, **na** number of ADCs to read and **nd** the read of digital I/O lines. On those commands, ADCs are read in order starting on ADC1. 

This command always read a single ADC channel, indicated on the **PC payload**, regardless of the current **na** and **nd** values.

Before starting the measurement, the indicated channel is routed, using the ADC multiplexers, to one hardware ADC peripheral which is the only one used during the measurement.

That way, all ADC channels are available to this command.

The command is designed to implement **alternate captures**. Through several calls to this command we can obtain the response of several ADCs to the same waveform. As only one ADC is sampled each time, the code can be optimized for speed. This can be handy for obtaining frequency responses at the maximum sample rate.

No digital signals are generated or measured on this command.

The rest of the command operation is similar to the **Wave Response** one.

**Example:** singleWaveResponse

<a id='WavePlay'></a>

## Wave Play (Q)

Plays a wave, without making measurements, on **DAC1**. It also generates digital waves on the I/O lines if a wavetable was loaded for them.

>PC
>* Send Char 'Q'
>* Send Word **wn**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Generate Wave
>* Send Byte **Tran Status**
>* Send Byte CRC

This command sends **wn** primary waveform waves to **DAC1**. If **wn** is zero, the waves will be generated with no end. In this case, the normal way to stop the wave generation is to use the **halt** button.

![Wave Play](images/Firmware/waveplay.png)

If a digital wavetable has been loaded, digital waves will also be generated.

If there is no **primary wavetable**, the **wn** value will refer to the digital waves.

The **Tran Status** codes are the same as in the **Async Read**

**Example:** wavePlay

<a id='DualWavePlay'></a>

## Dual Wave Play (q)

Plays two simultaneous waves, without making measurements, on **DAC1** and **DAC2**. It also generates digital waves on the I/O lines if a wavetable was loaded for them.

>PC
>* Send Char 'q'
>* Send Word **wn**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Generate Waves
>* Send Byte **Tran Status**
>* Send Byte CRC

This command is like the previous **Wave Play** command, but two waves are generated instead of one.

It is **ilegal** yo call this command if the **primary** and **secondary** wavetables have not been uploaded

**Example:** wavePlay with True **dual** parameter

<a id='DigitalIOMode'></a>

## Digital I/O mode (H)

Configures a Digital I/O line

>PC
>* Send Char 'H'
>* Send Byte **line**
>* Send Byte **mode** 
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Byte CRC

Configures the Digitsl I/O line **line** on mode **mode**

The **line** parameter is a number from 0 to **ndio** - 1

The mode can be one of the following:

* DMODE_INPUT : Normal input
* DMODE_PULLUP : Input with Pull-Up
* DMODE_PULLDOWN : Input with Pull-Down
* DMODE_OUTPUT : Normal Push-Pull output
* DMODE_OPENDRAIN : Open Drain output

Those modes shall be available on ST boards. In a different particular board is possible for some of the modes to be unavailable

If an ilegal **line** or **mode** is received, a **NACK** action takes place

**Example:** dioMode

<a id='DigitalIOWrite'></a>

## Digital I/O Write (J)

Sets the value of a Digital I/O line

>PC
>* Send Char 'J'
>* Send Byte **line**
>* Send Byte **value** 
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Byte CRC

If **value** is zero, sets DIO number **line** **Low**, if not, sets it **High**

If an ilegal **line** is received, a **NACK** action takes place

If the **line** was in input mode, the function has no inmediate effect but the value is remembered for the next change to output mode.

**Example:** dioWrite

<a id='DigitalIORead'></a>

##  Digital I/O Read (K)

Reads the value of a Digital I/O line

>PC
>* Send Char 'J'
>* Send Byte **line**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Byte **value**
>* Send Byte CRC

If an ilegal **line** is received, a **NACK** action takes place

If **line** is High, the returned **value** is 1, if not, the returned **value** is 0

The command reads the output pin state both if the **line** is configured as input or output.

**Example:** dioRead

<a id='DigitalIOWriteAll'></a>

## Digital I/O Write All (j)

Sets the value of all Digital I/O lines at the same time

>PC
>* Send Char 'j'
>* Send Word **value** 
>* Send Word **mask**
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Byte CRC

Set all digital I/O lines as indicated on **value**, with one bit of this word for each line

If **mask** is not zero, only the lines whose mask bits are set will be modified

If one I/O **line** is in input mode, the function has no inmediate effect but the value is remembered for the next change to output mode.

**Example:** dioWriteAll

<a id='DigitalIOReadAll'></a>

##  Digital I/O Read All (k)

Reads the value of all Digital I/O lines

>PC
>* Send Char 'j'
>* Send Byte CRC

>MCU
>* Send Byte ACK
>* Send Word **value**
>* Send Byte CRC

Reads all digital I/O lines with one bit in **value** for each I/O line

The command reads the output pin states both if the lines are configured as input or output.

**Example:** dioReadAll

<a id='CodeConstants'></a>

# Code constants

Those are the current values of the constants used in the communication protocol. All of then are transmitted in a single Byte

* ACK  181
* NACK  226
* ECRC  37

* TRAN_OK  0
* TRAN_OVERRUN  1
* TRAN_TIMEOUT  2
* TRAN_HALT  3

* TMODE_RISE 0
* TMODE_FALL 1

* DMODE_INPUT 10
* DMODE_PULLUP 11
* DMODE_PULLDOWN 12
* DMODE_OUTPUT 20
* DMODE_OPENDRAIN 21

The **Magic Code** used for the SLab firmware has 4 Bytes 56,41,18,1 that are sent in this order. 

## Document license

Copyright  ©  Vicente Jiménez (2018-2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">